# **IMDB Movie Review Classification:**
### Thirumurugan Ramar

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

TensorFlow 2.x selected.


In [2]:
tf.__version__

'2.1.0-rc1'

In [3]:
# Load IMDB reviews dataset fom tensorflow
tfds.load(name='imdb_reviews')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


{'test': <_OptionsDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <_OptionsDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <_OptionsDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

In [0]:
(train_data, validation_data), test_data = tfds.load(name='imdb_reviews',
         split=(tfds.Split.TRAIN.subsplit([6,4]),tfds.Split.TEST),
         as_supervised=True)

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [6]:
 train_labels_batch 

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [0]:
# Using pretrained models from tensorflow hub,to convert movie reviews to vectors
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [9]:
train_examples_batch[:2] #original

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by 

In [10]:
hub_layer(train_examples_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ]],
      dtype=float32)>

In [11]:
# Sequential Model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))    #hidden layer with 16 neurons
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
#fine tuning the model
model.fit(train_data.shuffle(10000).batch(512),
         epochs=20,
         validation_data=validation_data.batch(512),
         verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 213ms/step - loss: 0.7629 - accuracy: 0.5413 - val_loss: 0.6571 - val_accuracy: 0.6114
Epoch 2/20
30/30 [==============================] - 5s 182ms/step - loss: 0.6225 - accuracy: 0.6650 - val_loss: 0.6089 - val_accuracy: 0.6791
Epoch 3/20
30/30 [==============================] - 6s 187ms/step - loss: 0.5833 - accuracy: 0.7056 - val_loss: 0.5751 - val_accuracy: 0.7082
Epoch 4/20
30/30 [==============================] - 6s 201ms/step - loss: 0.5476 - accuracy: 0.7398 - val_loss: 0.5434 - val_accuracy: 0.7370
Epoch 5/20
30/30 [==============================] - 6s 198ms/step - loss: 0.5138 - accuracy: 0.7651 - val_loss: 0.5135 - val_accuracy: 0.7570
Epoch 6/20
30/30 [==============================] - 6s 186ms/step - loss: 0.4810 - accuracy: 0.7875 - val_loss: 0.4876 - val_accuracy: 0.7740
Epoch 7/20
30/30 [==============================] - 6s 195ms/step - loss: 0.4500 - accuracy: 0.8082 - val_loss: 0.4600 - val_accuracy: 0.7918
Epoch 

# ***The accuracy of the model is 94%*!**

### **let'S TEST the model:**

In [14]:
model.predict(["The Last Jedi is not only a fantastic film, but one of the best Star Wars films."])

array([[0.9192961]], dtype=float32)

In [16]:
model.predict(["The Justice league is a pointless flail of expensive (yet, somehow, cheap-looking) CGI that no amount of tacked-on quips"])

array([[0.04450963]], dtype=float32)

In [21]:
model.predict(["Kaidhi is a Masterpiece and is the best in Karthi's career."])

array([[0.88209504]], dtype=float32)